# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [8]:
!cat publications.tsv

## Import pandas

We are using the very handy pandas library for dataframes.

In [9]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [10]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url
0,2019-06-13,Collaborative Writing at Scale: A Case Study o...,ACM Collective Intelligence (CI'19),Work of all kinds is increasingly done in a ne...,"Pe-Than, E.P.P., Dabbish, L., and Herbsleb, J....",CI-poster-2019,http://eipapa.github.io/files/CI-poster-2019.pdf
1,2019-06-25,Collaborative Writing at Scale: A Case Study o...,The Future of Work (HCIC'19) Workshop,Collaborative Work of all kinds is increasingl...,"Pe-Than, E.P.P, Dabbish, L., and Herbsleb, J.D...",HCIC-2019,http://eipapa.github.io/files/HCIC-poster-2019...
2,2018-07-11,Designing Corporate Hackathons with a Purpose,IEEE Software,"In hackathons, small teams work together over ...","Pe-Than, E.P.P., Nolte, A., Filippova, A., Bir...",IEEESW-2018,http://eipapa.github.io/files/IEEESW-2018.pdf
3,2018-11-01,You Hacked and Now What?: - Exploring Outcomes...,Proceedings of the ACM on Human-Computer Inter...,"Time bounded events such as hackathons, data d...","Nolte, A., Pe-Than, E.P.P., Filippova, A., Bir...",CSCW-2018,http://eipapa.github.io/files/CSCW-2018.pdf
4,2019-03-13,Understanding Hackathons for Science: Collabor...,iConference'19,"Nowadays, hackathons have become a popular way...","Pe-Than, E.P.P. and Herbsleb, J.D. (2019). Und...",iConference-2019,http://eipapa.github.io/files/iConference-2019...
5,2018-10-01,Collaborative Writing on GitHub: A Case Study ...,Companion of the 2018 ACM Conference on Comput...,Social coding platforms such as GitHub are inc...,"Pe-Than, E.P.P., Dabbish, L., and Herbsleb, J....",CSCW-abstract-2018,http://eipapa.github.io/files/CSCW-abstract-20...
6,2018-01-04,The 2nd Workshop on Hacking and Making at Time...,Extended Abstracts of the 2018 ACM CHI Confere...,"Hackathons or Hackathon-style events, describe...","Pe-Than, E.P.P., Herbsleb, J.D., Nolte, A., Ge...",CHI-workshop-2018,http://eipapa.github.io/files/CHI-workshop-201...
7,2019-01-01,"Hackathons for Science, How and Why?",The 233rd Meeting of the American Astronomical...,Based on our empirical studies of 14 hackathon...,"Pe-Than, E.P.P., Momcheva, I., Tollerud, E., a...",AAS-poster-2019,http://eipapa.github.io/files/AAS-poster-2019.pdf
8,2018-01-09,An Analysis of the PEARC 2018 Science Gateways...,Practice & Experience in Advanced Research Com...,Science Gateways are web portals on which data...,"Powell, J., Hayden, L., Nolte, A., Herbsleb, J...",SGCI-poster-2018,http://eipapa.github.io/files/SGCI-poster-2018...
9,2018-12-30,The 2nd Workshop on Hacking and Making at Time...,Technical Report CMU-ISR-18-109,"In hackathons, small teams work together over ...","Pe-Than, E.P.P., and Nolte, A. (Editors) (2018...",ISR-techreport-2018,http://eipapa.github.io/files/ISR-techreport-2...


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [11]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [12]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md = "layout: archive" + '"\n'
    md += """collection: publications"""
    md += """\npermalink: /publication/""" + html_filename
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    md += "\ndate: " + str(item.pub_date) 
    md += "\nvenue: '" + html_escape(item.venue) + "'"    
#     if len(str(item.paper_url)) > 5:
#         md += "\npaperurl: '" + item.paper_url + "'"
#     md += "\ncitation: '" + html_escape(item.citation) + "'"
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.paper_url)) > 5:
#         md += "\n[<span style=\"color: #c41e3a\">Download PDF here.</span>](" + item.paper_url + ")\n"
        md += "\n[Download PDF here.](" + item.paper_url + ")\n"
    if len(str(item.excerpt)) > 5:
        md += "\n**Abstract**: " + html_escape(item.excerpt) + "\n"    
#     md += "\nAbstract: " + html_escape(item.description) + "\n"
    md += "\n**Recommended citation**: " + item.citation
    md_filename = os.path.basename(md_filename)
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [13]:
!ls ../_publications/

2018-01-04-CHI-workshop-2018.md   2019-01-01-AAS-poster-2019.md
2018-01-09-SGCI-poster-2018.md    2019-03-13-iConference-2019.md
2018-07-11-IEEESW-2018.md         2019-06-13-CI-poster-2019.md
2018-10-01-CSCW-abstract-2018.md  2019-06-25-HCIC-2019.md
2018-11-01-CSCW-2018.md           backup
2018-12-30-ISR-techreport-2018.md


In [ ]:
!cat ../_publications/2009-10-01-paper-title-number-1.md